In [1]:
import json
import pickle
import gzip
import re
from collections import defaultdict, Counter
import pymorphy3
import requests
from bs4 import BeautifulSoup
import time
import json
import os
from urllib.parse import urljoin
from abc import ABC, abstractmethod
from functools import wraps
import logging
import unittest

In [2]:
def setup_logging():
    log_format = '%(asctime)s - [%(levelname)s] - %(name)s - %(message)s'
    
    logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    handlers=[
        logging.FileHandler('stichi_log.log', encoding='utf-8')
    ]
)

In [3]:
setup_logging()

In [4]:
logger = logging.getLogger(__name__)

In [5]:
def log_processing(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        logger = logging.getLogger(self.__class__.__module__ + '.' + self.__class__.__name__)
        
        poem = None
        for arg in args:
            if hasattr(arg, 'title'):
                poem = arg
                break
        
        title = poem.title if poem else "Неизвестно"
        logger.info(f"Начата обработка: {title}")
        start_time = time.time()
        
        try:
            result = func(self, *args, **kwargs)
            elapsed = time.time() - start_time
            logger.info(f"Успешно завершено: {title} за {elapsed:.2f}с")
            return result
        except Exception as e:
            logger.error(f"Ошибка при обработке {title}: {e}", exc_info=True)
            raise
    return wrapper

In [6]:
class PoemAnalysisError(Exception):
    def __init__(self, message):
        logger.error(f"PoemAnalysisError: {message}")
        super().__init__(message)

class EmptyPoemError(PoemAnalysisError): #пустое стихотворение
    pass

class DataFetchError(PoemAnalysisError): #ошибка загрузки данных
    pass

In [7]:
class DataFetchError(PoemAnalysisError):
    def __init__(self, message):
        logger = logging.getLogger(__name__)
        logger.error(f"DataFetchError: {message}")
        super().__init__(message)

In [8]:
class Poem:
    def __init__(self, title, lines, url=""):
        self.title = title
        self.lines = lines
        self.url = url
        self.meter = None
        self.rhyme_scheme = None

    def __str__(self):
        return f"{self.title} ({len(self.lines)} строк)"

    def __len__(self):
        return len(self.lines)

    def is_empty(self):
        return len([l for l in self.lines if l.strip()]) == 0

    @property
    def text(self):
        return "\n".join(self.lines)

In [9]:
class VerseProcessor(ABC):

    def __init__(self, language="ru"):
        self.language = language

    @abstractmethod
    def process(self, poem): #обработка стихотворения
        pass

    def validate_input(self, poem): # проверка данных
        if not poem or poem.is_empty():
            raise EmptyPoemError("Стихотворение пустое")
        return True

In [10]:
class MetricalAnalyzer(VerseProcessor):

    def __init__(self):
        super().__init__("ru")
        self.morph = pymorphy3.MorphAnalyzer(lang='ru')
        self.meters = {
            'ямб': [0, 1],
            'хорей': [1, 0],
            'дактиль': [1, 0, 0],
            'амфибрахий': [0, 1, 0],
            'анапест': [0, 0, 1]
        }
        logger.debug(f"Инициализирован MetricalAnalyzer")

    @log_processing
    def process(self, poem): #метр
        logger.debug(f"Начинаем анализ метра для {poem.title}")
        self.validate_input(poem)

        meter_scores = defaultdict(int)
        total_lines = 0

        for line in poem.lines:
            if line.strip():
                total_lines += 1
                line_meter = self._analyze_line(line)
                if line_meter:
                    meter_scores[line_meter] += 1
                    logger.debug(f"Строка: {line_meter}")

        #основной метр
        if meter_scores and total_lines > 0:
            dominant = max(meter_scores, key=meter_scores.get)
            confidence = meter_scores[dominant] / total_lines
            poem.meter = dominant if confidence > 0.7 else "смешанный"
            logger.info(f"Результат анализа {poem.title}: {poem.meter}, уверенность: {confidence:.2%}")
            return {"meter": poem.meter, "confidence": confidence}
        else:
            poem.meter = "неопределен"
            logger.warning(f"Метр не определён для {poem.title}")
            return {"meter": "неопределен", "confidence": 0.0}



    def _analyze_line(self, line):
        logger.debug(f"Анализ строки: '{line[:50]}...'")
        bitstring = self._line_to_bitstring(line) #битовая строка ударений

        if not bitstring:
            logger.debug("Пустая битовая строка")
            return None

        #сравнение с эталонами
        scores = {}
        for meter_name, meter_pattern in self.meters.items():
            score = self._compare_with_meter(bitstring, meter_pattern)
            scores[meter_name] = score

        best_meter = max(scores, key=scores.get)
        return best_meter if scores[best_meter] > 0.5 else None

    def _line_to_bitstring(self, line):
        bitstring = []
        i = 0
        vowels = "аеёиоуыэюя"
        while i < len(line):
            char = line[i]
            if char in "`'~": #исключение для ударения
                i += 1
                continue
                
            #обработка ударений
            if char.lower() in vowels:
                if i + 1 < len(line) and line[i + 1] in "`'~":
                    bitstring.append('1')
                    i += 2
                else:
                    bitstring.append('0')
                    i += 1
            else:
                i += 1
        return ''.join(bitstring)

    def _compare_with_meter(self, bitstring, meter_pattern):
        if not bitstring:
            return 0.0

        pattern_length = len(meter_pattern)
        meter_bitstring = ''.join(str(meter_pattern[i % pattern_length])
                                  for i in range(len(bitstring)))

        #индекс жаккара
        matches = sum(1 for a, b in zip(bitstring, meter_bitstring) if a == '1' and b == '1')
        total_emphasised = sum(1 for b in bitstring if b == '1')

        if total_emphasised == 0:
            return 0.0
        return matches / total_emphasised

In [11]:
class DataCollector:

    def __init__(self, base_url="https://rvb.ru/pushkin/"):
        self.base_url = base_url
        self.session = requests.Session()
        self.logger = logging.getLogger(f"{__name__}.DataCollector")
        self.logger.info(f"Инициализация DataCollector с base_url={base_url}")

    def fetch_poem(self, url):
        try:
            response = self.session.get(url, timeout=10)
            response.encoding = "windows-1251"

            soup = BeautifulSoup(response.text, "html5lib")
            title = self._extract_title(soup)
            lines = self._extract_lines(soup)

            return Poem(title=title, lines=lines, url=url)

        except requests.RequestException as e:
            self.logger.error(f"Ошибка загрузки {url}: {e}", exc_info=True)
            raise DataFetchError(f"Ошибка загрузки {url}: {e}")

    def _extract_title(self, soup):
        h1 = soup.find("h1")
        return h1.get_text(strip=True) if h1 else "Без заголовка"

    def _extract_lines(self, soup):
        lines = []
        for stanza in soup.find_all("p", class_=["stanza", "continuation"]):
            for span in stanza.find_all("span", class_=["line", "line2r"]):
                lines.append(span.get_text(strip=True))
            lines.append("")  #пустая строка между строфами

        return lines[:-1] if lines and lines[-1] == "" else lines

In [12]:
class PoemCollection:
    #перегрузка
    def __init__(self):
        self.poems = []
        self._index_by_title = {}

    def add_poem(self, poem):
        if not poem.is_empty():
            self.poems.append(poem)
            self._index_by_title[poem.title] = poem
            return True
        else:
            print(f"Пустое: {poem.title}")
            return False
            
    def __len__(self):
        return len(self.poems)

    def __getitem__(self, idx):
        if isinstance(idx, int):
            return self.poems[idx]
        elif isinstance(idx, str):
            return self._index_by_title.get(idx)
        elif isinstance(idx, slice):
            return self.poems[idx]
        else:
            raise TypeError("Индекс должен быть int, slice или str")

    def __iter__(self):
        return iter(self.poems)

    def __contains__(self, item):
        if isinstance(item, Poem):
            return item in self.poems
        elif isinstance(item, str):
            return item in self._index_by_title
        return False


    def save_to_json(self, filename):
        data = []
        for poem in self.poems:
            data.append({
                "title": poem.title,
                "lines": poem.lines,
                "url": poem.url,
            })

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"Коллекция сохранена в {filename}")

    def get_statistics(self):
        meters_count = Counter(p.meter for p in self.poems if p.meter)
        return {
            "total_poems": len(self),
            "total_lines": sum(len(p) for p in self.poems),
            "meters": dict(meters_count),
            "avg_lines_per_poem": sum(len(p) for p in self.poems) / len(self) if self.poems else 0
        }


In [13]:
class PoemFactory:

    @staticmethod
    def from_json(filepath):
        collection = PoemCollection()
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for item in data:
                poem = Poem(
                    title=item.get('title', ''),
                    lines=item.get('lines', []),
                    url=item.get('url', ''),
                )
                collection.add_poem(poem)
        return collection

    @classmethod
    def from_urls(cls, urls, collector):
        collection = PoemCollection()
        for i, url in enumerate(urls, 1):
            print(f"[{i}/{len(urls)}] Загрузка {url}")
            try:
                poem = collector.fetch_poem(url)
                collection.add_poem(poem)
                time.sleep(0.3)  #вежливость
            except Exception as e:
                print(f"Ошибка при загрузке {url}: {e}")
        return collection

юниттесты

In [14]:
class TestLineToBitstring(unittest.TestCase):
    
    def setUp(self):
        self.analyzer = MetricalAnalyzer()

    def test_simple(self):
        self.assertEqual(self.analyzer._line_to_bitstring("промча`лся"), "010")
        self.assertEqual(self.analyzer._line_to_bitstring("промчался"), "000")
        self.assertEqual(self.analyzer._line_to_bitstring("ве`тер"), "10")

    def test_multiple(self):
        self.assertEqual(self.analyzer._line_to_bitstring("крова`вый ме`ч"), "0101")
        self.assertEqual(self.analyzer._line_to_bitstring("крова`вый ме`ч"), "0001") #ошибка
    
    def test_no_vowels(self):
        self.assertEqual(self.analyzer._line_to_bitstring("втр"), "")
      

In [15]:
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_multiple (__main__.TestLineToBitstring.test_multiple) ... FAIL
test_no_vowels (__main__.TestLineToBitstring.test_no_vowels) ... ok
test_simple (__main__.TestLineToBitstring.test_simple) ... ok

FAIL: test_multiple (__main__.TestLineToBitstring.test_multiple)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\savch\AppData\Local\Temp\ipykernel_16596\3325843395.py", line 13, in test_multiple
    self.assertEqual(self.analyzer._line_to_bitstring("крова`вый ме`ч"), "0001") #ошибка
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: '0101' != '0001'
- 0101
?  ^
+ 0001
?  ^


----------------------------------------------------------------------
Ran 3 tests in 0.164s

FAILED (failures=1)


In [16]:
def parse_pushkin_volume1():

    base_url = "https://rvb.ru/pushkin/"
    toc_url = "https://rvb.ru/pushkin/tocvol1.htm"
    output_file = "pushkin_vol1.json"

    collector = DataCollector(base_url)

    #список ссылок
    print("Получаю список стихотворений...")
    poem_links = get_poem_links(toc_url, collector)

    if not poem_links:
        print("Не найдено ссылок на стихотворения")
        return None

    print(f"Найдено ссылок: {len(poem_links)}")

    #через PoemFactory
    print("\nЗагружаю стихотворения...")
    collection = PoemFactory.from_urls(poem_links, collector)

    print(f"Успешно загружено: {len(collection)} стихотворений")

    print(f"\nСохраняю в {output_file}...")
    collection.save_to_json("output.json")

    print(f"Готово! Файл сохранён: {os.path.abspath(output_file)}")

    return collection

def get_poem_links(toc_url, collector):
    try:
        response = collector.session.get(toc_url, timeout=10)
        response.encoding = "windows-1251"
        soup = BeautifulSoup(response.text, "html5lib")

        poem_links = []
        for table in soup.find_all("table"):
            for a in table.find_all("a", href=True):
                href = a["href"]

                if any(x in href for x in ["articles", "comm", "index", "txtindex", "adresindex", "nameindex"]):
                    continue
                if href.startswith("http"):
                    continue

                full_url = urljoin(toc_url, href)
                poem_links.append(full_url)

        return poem_links

    except Exception as e:
        print(f"Ошибка при получении ссылок: {e}")
        return []

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_poems, f, ensure_ascii=False, indent=2)

In [17]:
if __name__ == "__main__":
    collection = parse_pushkin_volume1()

Получаю список стихотворений...
Найдено ссылок: 326

Загружаю стихотворения...
[1/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0001.htm
[2/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0002.htm
[3/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0003.htm
Пустое: СТАРИК
[4/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0004.htm
Пустое: РОЗА
[5/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0005.htm
[6/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0006.htm
[7/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0007.htm
Пустое: К МОРФЕЮ
[8/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0008.htm
Пустое: ДРУЗЬЯМ
[9/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0009.htm
[10/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0010.htm
[11/326] Загрузка https://rvb.ru/pushkin/01text/01versus/01lyceum/0011.htm
Пустое: ПРОБУЖДЕНИЕ
[12/326] Загрузка https:

словарь ударений

In [18]:
# исправленое!!
def parse_par_file(filepath):
    emphasis_dict = defaultdict(lambda: defaultdict(set))

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            #НЕБО1
            if re.match(r'^[А-ЯЁ0-9$]+$', line):
                continue

            #не`бо - S,ЕД,...
            if '-' not in line:
                continue

            parts = line.split('-', 1)
            emphasised_form = parts[0].strip()
            pos_info = parts[1].strip()

            pos = pos_info.split(',')[0]  #метка POS

            #ударения, ё - е, нижний регистр
            lemma_like = re.sub(r'[`\~]', '', emphasised_form).lower()
            lemma_like = lemma_like.replace('ё', 'е')

            #форма с ударением в нижнем регистре
            emphasised_form_lower = emphasised_form.lower()

            emphasis_dict[lemma_like][pos].add(emphasised_form_lower)

    #в dict
    return {word: {pos: forms for pos, forms in pos_dict.items()}
            for word, pos_dict in emphasis_dict.items()}

In [19]:
def save_dict_to_pkl_gz(data, output_path):
    with gzip.open(output_path, 'wb') as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
def load_emphasis_dict(path="emphasis_dict.pkl.gz"):
    with gzip.open(path, "rb") as f:
        return pickle.load(f)

In [20]:
if __name__ == "__main__":
    input_file = "par.txt"
    output_file = "emphasis_dict.pkl.gz"

    print("Парсинг файла...")
    emphasis_dict = parse_par_file(input_file)

    save_dict_to_pkl_gz(emphasis_dict, output_file)
    print(f"Словарь сохранён в {output_file}")

Парсинг файла...
Словарь сохранён в emphasis_dict.pkl.gz


Словарь ё

In [21]:
class Yoficator:
    def __init__(self, dict_path="yo_dict.pkl.gz"):
        #словарь из .pkl.gz файла.

        with gzip.open(dict_path, 'rb') as f:
            self.yo_dict = pickle.load(f)
        print(f"Ёфикатор загружен: {len(self.yo_dict):,} слов.")

    def _restore_case(self, original: str, replacement: str) -> str: #регистр

        if original.isupper():
            return replacement.upper()
        elif original.istitle():
            return replacement.capitalize()
        else:
            return replacement

    def yoficate_word(self, word: str) -> str:
        lower_word = word.lower()
        if lower_word in self.yo_dict:
            corrected = self.yo_dict[lower_word]
            return self._restore_case(word, corrected)
        return word

In [22]:
def build_yo_normalization_dict(par_file_path):

    #словарь замены слов с 'е' на 'ё' на основе par.txt. без (` и ~)

    yo_dict = {}

    with open(par_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if not line or '$' in line:
            continue

        match = re.match(r'^(.+?)\s*-\s*[A-Z]+', line)
        if not match:
            continue

        emphasised_form = match.group(1).strip()

        #` и ~
        clean_form = emphasised_form.replace('`', '').replace('~', '')

        clean_lower = clean_form.lower()

        if 'ё' not in clean_lower:
            continue

        variant_with_e = clean_lower.replace('ё', 'е')

        if variant_with_e != clean_lower:
            yo_dict[variant_with_e] = clean_lower

    return yo_dict

In [23]:
def save_yo_dict(yo_dict, output_path):
    with gzip.open(output_path, 'wb') as f:
        pickle.dump(yo_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_yo_dict(path):
    with gzip.open(path, 'rb') as f:
        return pickle.load(f)

In [24]:
if __name__ == "__main__":
    par_file = "par.txt"
    output_file = "yo_dict.pkl.gz"

    print("Строим словарь из par.txt...")
    yo_dict = build_yo_normalization_dict(par_file)

    print(f"Словарь содержит {len(yo_dict):,} пар.\n")

    save_yo_dict(yo_dict, output_file)
    print(f"\nСловарь сохранён в {output_file}")

Строим словарь из par.txt...
Словарь содержит 121,796 пар.


Словарь сохранён в yo_dict.pkl.gz


Обработка

In [25]:
yoficator = Yoficator("yo_dict.pkl.gz")

Ёфикатор загружен: 121,796 слов.


In [26]:
def add_emphasis_to_word(word, emphasis_dict, yoficator):
    yo_word = yoficator.yoficate_word(word)

    #нормализую: ё - е (если в словаре ключи с "е")
    search_key = yo_word.lower().replace('ё', 'е')

    if search_key in emphasis_dict:
        #берем первую форму с ударением
        for pos_forms in emphasis_dict[search_key].values():
            for emphasised_form in pos_forms:
                #восстанавливаю регистр
                if word.isupper():
                    return emphasised_form.upper()
                elif word.istitle():
                    return emphasised_form.capitalize()
                else:
                    return emphasised_form

    return yo_word

def emphasize_text(text, emphasis_dict, yoficator):
    def process_match(match):
        word = match.group(0)
        return add_emphasis_to_word(word, emphasis_dict, yoficator)

    return re.sub(r'[А-Яа-яЁё]+', process_match, text)

In [27]:
processed_poems = []
with open('pushkin_vol1.json', 'r', encoding='utf-8') as f:
    all_poems = json.load(f)

for i, poem in enumerate(all_poems):
    print(f"Обработка [{i+1}/{len(all_poems)}]: {poem['title'][:50]}...")

    processed_lines = []
    for line in poem['lines']:
        if line.strip():
            processed_line = emphasize_text(line, emphasis_dict, yoficator)
            processed_lines.append(processed_line)
        else:
            processed_lines.append("")

    processed_poems.append({
        "title": poem['title'],
        "lines": processed_lines,
        "url": poem['url']
    })
processed_output_file = "pushkin_vol1_processed.json"
with open(processed_output_file, 'w', encoding='utf-8') as f:
    json.dump(processed_poems, f, ensure_ascii=False, indent=2)

print(f"Сохранено в {processed_output_file}")

Обработка [1/131]: ВОСПОМИНАНИЯ В ЦАРСКОМ СЕЛЕ...
Обработка [2/131]: ЛИЦИНИЮ...
Обработка [3/131]: ГРОБ АНАКРЕОНА...
Обработка [4/131]: ПЕВЕЦ...
Обработка [5/131]: АМУР И ГИМЕНЕЙ...
Обработка [6/131]: ШИШКОВУ...
Обработка [7/131]: К ОГАРЕВОЙ,КОТОРОЙ МИТРОПОЛИТ ПРИСЛАЛ ПЛОДОВ ИЗ СВ...
Обработка [8/131]: ТУРГЕНЕВУ...
Обработка [9/131]: К ***...
Обработка [10/131]: ВОЛЬНОСТЬОДА...
Обработка [11/131]: ТОРЖЕСТВО ВАКХА...
Обработка [12/131]: ВЫЗДОРОВЛЕНИЕ...
Обработка [13/131]: СКАЗКИNOËL...
Обработка [14/131]: ПРЕЛЕСТНИЦЕ...
Обработка [15/131]: N. N....
Обработка [16/131]: ОРЛОВУ...
Обработка [17/131]: КЩЕРБИНИНУ...
Обработка [18/131]: ДЕРЕВНЯ...
Обработка [19/131]: РУСАЛКА...
Обработка [20/131]: НЕДОКОНЧЕННАЯ КАРТИНА...
Обработка [21/131]: ВСЕВОЛОЖСКОМУ...
Обработка [22/131]: ПЛАТОНИЗМ...
Обработка [23/131]: СТАНСЫТОЛСТОМУ...
Обработка [24/131]: ВОЗРОЖДЕНИЕ...
Обработка [25/131]: ПОСЛАНИЕ К кн.ГОРЧАКОВУ...
Обработка [26/131]: ЮРЬЕВУ...
Обработка [27/131]: * * *...
Обработка [28/131]: ЧЕРНА

In [28]:
def main():
    setup_logging()
    logger = logging.getLogger(__name__)
    
    logger.info("Автоматическое определение стихотворного метра")
    
    try:
        factory = PoemFactory()
        collection = factory.from_json(processed_output_file)
        logger.info(f"Загружено {len(collection)} стихотворений")
        
        analyzer = MetricalAnalyzer()
        
        results = []
        
        for i, poem in enumerate(collection, 1):
            logger.info(f"[{i}/{len(collection)}] Анализируем: {poem.title}")
            result = analyzer.process(poem)
            logger.debug(f"Результат: {result}")
            
            results.append({
                "title": poem.title,
                "meter": result.get("meter", "неопределен"),
                "confidence": round(result.get("confidence", 0.0), 4),
                "url": poem.url,
                "line_count": len(poem)
            })
        stats = collection.get_statistics()
        logger.info(f"Статистика: {stats}")

        print(f"Всего стихотворений: {stats['total_poems']}")
        print(f"Всего строк: {stats['total_lines']}")
        print(f"Среднее строк на стихотворение: {stats['avg_lines_per_poem']:.1f}")
        print("\nРаспределение метров:")
        
        for meter, count in stats['meters'].items():
            percentage = (count / stats['total_poems']) * 100
            print(f"  {meter}: {count} ({percentage:.1f}%)")

        results_file = "results.json"
        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump({
                "metadata": {
                    "analysis_date": time.strftime("%Y-%m-%d %H:%M:%S"),
                    "total_poems": len(collection),
                    "total_lines": stats["total_lines"]
                },
                "statistics": stats,
                "detailed_results": results
            }, f, ensure_ascii=False, indent=2, default=str)

        logger.info(f"Результаты сохранены в {results_file}")
        
    except Exception as e:
        logger.critical(f"Критическая ошибка в main(): {e}", exc_info=True)
        raise
    
    logger.info("Завершено успешно")
    logging.shutdown()


In [29]:
if __name__ == "__main__":
    main()

Всего стихотворений: 131
Всего строк: 4139
Среднее строк на стихотворение: 31.6

Распределение метров:
  ямб: 106 (80.9%)
  хорей: 11 (8.4%)
  смешанный: 11 (8.4%)
  неопределен: 3 (2.3%)


Переделано под критерии:

1. Использование классов.
 Poem, VerseProcessor, MetricalAnalyzer, DataCollector, PoemCollection, PoemFactory, Yoficator
2. Использование наследования от какого-нибудь класса.
 MetricalAnalyzer наследуется от VerseProcessor
3. Использование наследования от созданного своими руками класса.
 MetricalAnalyzer наследуется от VerseProcessor
4. Использование абстрактных классов в структуре классов.
VerseProcessor
5. Три библиотеки из тех, что приходится ставить своими руками.
pymorphy3, beautifulsoup4, requests
6. Использование декораторов.
@abstractmethod, @log_processing, @wraps
7. Использование собственного декоратора.
@log_processing
8. Перегрузка операторов.
в PoemCollection
10. Использование каких-либо исключений.
PoemAnalysisError, EmptyPoemError, DataFetchError
11. Использование исключений своего собственного типа.
PoemAnalysisError, EmptyPoemError, DataFetchError